In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt 
import copy
import timeit
import time
import subprocess
import pickle
def plotp (m):
    data=m[::-1,:]
    heatmap = plt.pcolor(data, cmap=plt.cm.tab20, edgecolors='grey', linewidths=2)
    for y in range(data.shape[0]):
        for x in range(data.shape[1]):
            plt.text(x + 0.5, y + 0.5, '%.0f' % data[y, x],
                     horizontalalignment='center',
                     verticalalignment='center',
                     )
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.show()
def start (w,h):
    arr = np.zeros(w*h,dtype=int) 
    arr[:random.randrange(0, w*h+1)]  = 1
    np.random.shuffle(arr)
    g = np.reshape(arr,(w,h))
    return g
g=start(5,5)

In [ ]:
def balancednessDetection(field):
    def rectangleRecognition(field):
        st = timeit.default_timer()
        for i in range(len(field)-1):
            for j in range(len(field[0])-1):
                if field[i,j]>0:
                    if field[i+1,j]>0 and field[i,j+1]>0 and field[i+1,j+1]>0:
                        if (i+2 <len(field) and field[i+2,j]>0 and field[i+2,j+1]>0) or (j+2 < len(field[0]) and field[i,j+2]>0 and field[i+1,j+2]>0):
                            return True
        return False
    def squaresRecognition(field):
        st = timeit.default_timer()
        for i in range(1,len(field)-1):
            for j in range(1,len(field[0])-1):
                if field[i,j-1]==1 and field[i-1,j]==1 and field[i,j]==1 and field[i+1,j]==1 and field[i,j+1]==1 and ((field[i-1,j+1]==1 and field[i+1,j-1]==1) or (field[i-1,j-1]==1 and field[i+1,j+1]==1)):
                    return True
        return False
    def cycles(field):
        st = timeit.default_timer()
        pixels=set()
        w,h = len(field),len(field[0])
        for i in range(w):
            for j in range(h):
                if field[i,j]==1:
                    pixels.add((i,j))
        while pixels:
            root = pixels.pop()
            stack = [root]
            pred = {root: root}
            used = {root: set()}
            while stack:
                z = stack.pop()
                zused = used[z]
                q=[]
                i,j = z
                if i>0 and field[i-1,j]==1:
                    q.append((i-1,j))
                if i<w-1 and field[i+1,j]==1:
                    q.append((i+1,j))
                if j>0 and field[i,j-1]==1:
                    q.append((i,j-1))
                if j<h-1 and field[i,j+1]==1:
                    q.append((i,j+1))
                for nb in q:
                    if nb not in used:
                        pred[nb] = z
                        stack.append(nb)
                        used[nb] = {z}
                    elif nb == z: continue
                    elif nb not in zused:
                        pn = used[nb]
                        cycle = [nb, z]
                        p = pred[z]
                        while p not in pn:
                            cycle.append(p)
                            p = pred[p]
                        cycle.append(p)
                        if len(cycle)>4: 
                            return True
                        used[nb].add(z)
            pixels -= set(pred)
        return False
    if rectangleRecognition(field) and squaresRecognition(field) and cycles(field):
        return False
    else: return True

In [ ]:
def computeIPMatrix(field):

    def black(field):
        counter=0
        newField=copy.deepcopy(field)
        b=1
        for i in range(len(field)):
            for j in range(len(field[0])):
                if field[i,j]==1:
                    newField[i,j]=b
                    b+=1
                    counter+=1
        return newField,counter
        
    newField,b=black(field)
    w,h=len(field),len(field[0])
    rects=[]
    for i in range(0,w):
        for j in range(0,h):
            if field[i][j]!=0:
                d=h
                for ii in range(i,w):
                    for jj in range(j,d):
                        if field[ii][jj]==0: 
                            d=jj
                            break
                        if  i !=ii or j!=jj:
                            x=np.zeros(b)

                            for k in range(i,ii+1):
                                for kk in range(j,jj+1):
                                    x[newField[k,kk]-1]=1
                            rects.append(x)
    rects=np.array(rects) 
    rects=np.transpose(rects)
    if len(rects)>0:
        rects=rects[~np.all(rects == 0, axis=1)]
        if len(rects)<b:
            for i in range(b-len(rects)):
                rects= np.vstack((rects,np.zeros(len(rects[0]))))
        A = np.concatenate((rects,np.identity(b)),axis=1)
    else:
        A=np.identity(b)

    return A

field=[[1,1,0],
 [1,1,1],
 [0,1,1]]
field=np.reshape(field,(3,3))
plotp(field)
computeIPMatrix(field)

In [ ]:
def saveMatrix(field,n):
    lengths1 = len(field)
    lengths2 = len(field[0])
    st = "" + str(lengths2) + " " + str(lengths1)        
    f = open("matricesToTest/%s.mat" %n, 'w')
    f.write(st + "\n")
    for i in range(lengths1):
        st = "\n "
        for j in range(lengths2):
            st += str(int(field[i,j])) + " "
        f.writelines(st)

In [ ]:
def run(anz,size):

    fields = []
    matrices = []
    for i in range(anz):
        field = []
        matrix=[]
        while len(h) == 0:
            field = start(size, size)
            matrix = computeIPMatrix(field)
            
        saveMatrix(matrix,i)
        fields.append(field)
        matrices.append(matrix)
        print("Matrix", i)

    t = time.time()
    for field in fields: 
        balancednessDetection(field)
    print("Time our algorithm" , time.time()-t)

    #bashCommand = "bash ./run DT %s.mat" %i

    # bashCommand = "bash ./run DT ${SIZE}.mat"

    # process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    # output, error = process.communicate()
    # print(output)

    # x = []
    # for i in range(anz):
    #     # f1 = open("%s.mat-CE.log" %i, 'r')
    #     # string = f1.read().splitlines()
        
    #     # for j in string:
    #     #     s1 = float(j[5:9])
    #     # f1.close
    #     f2 = open("%s.mat-DT.log" %i, 'r')
    #     string = f2.read().splitlines()
    #     for j in string:
    #         s2 = float(j[5:9])
    #     f2.close
    #     # f3 = open("%s.mat-DT+V.log" %i, 'r')
    #     # string = f3.read().splitlines()
    #     # for j in string:
    #     #     s3 = float(j[5:9])
    #     # f3.close
    #     # f4 = open("%s.mat-ST.log" %i, 'r')
    #     # string = f4.read().splitlines()
    #     # s4 = np.infty
    #     # for j in string:
    #     #     if j != "Command terminated by signal 11":
    #     #         s4 = float(j[5:9])
    #     # f4.close

    #     #x.append(min(s1,s2,s3,s4))
    #     x.append(s2)
    
    # print("Time Matthias Walters algorithm" , sum(x))

In [ ]:
def runWaltersCode():

    bashCommand = "bash run-all"

    process = subprocess.Popen(bashCommand.split())
    output, error = process.communicate()
    out = str(output)

    #print(out)
    i = 0
    uni = []
    time = []
    while i < len(out):
        if out[i:i+6] == "matrix":
            while i < len(out):
                if out[i:i+3] == "not":
                    uni.append(0)
                    i += 25
                    break
                if out[i:i+3] == "tot":
                    uni.append(1)  
                    i += 25
                    break
                i+=1  
        if out[i:i+4] == "time":
            while out[i] not in ["0","1","2","3","4","5","6","7","8","9"]:
                i+=1
            ii=i
            while out[i] in ["0","1","2","3","4","5","6","7","8","9"]:
                i +=1
            time.append(float(out[ii:i+1])/1000000000)
        i +=1
    return time, uni

In [ ]:
stepsize = range(5,26,5)
anz = 100
wdh = 1

numUni = [[] for j in range(wdh)]
timeWalter = [[] for j in range(wdh)]
numBal = [[] for j in range(wdh)]
timeUs = [[] for j in range(wdh)]

timeIP = [[] for j in range(wdh)]

for x in stepsize:
    print("SIZE",x,"x",x)

    t = time.time()
    #fileObj2 = open('matrices%s.obj'%x, 'rb')
    #matrices = pickle.load(fileObj2)
    #fileObj2.close()

    fileObj1 = open('fields%s.obj'%x, 'rb')
    fields = pickle.load(fileObj1)
    fileObj1.close()

    # fields = []
    # matrices = []
    # t = time.time()
    for i in range(anz):
    #     g = []
         h=[]
         while len(h) == 0:
             g = start(x, x)
             h = computeIPMatrix(g)        
         saveMatrix(h,i)
    #     fields.append(g)
    #     matrices.append(h)
    
    print(x, "Loading of instances", time.time()-t)
    for z in range(wdh):
        print("Iteration:", z)

        tt = time.time()
        balanced = []
        timeU = []
        for j in range(len(fields)): 
            t = time.time()
            balanced.append(balancednessDetection(fields[j]))
            timeU.append(time.time()-t)
        print(z,x, "Time of our algorithm" , time.time()-tt, "Number of balanced fields:", sum(balanced))
        numBal[z].append(sum(balanced))
        timeUs[z].append(timeU)

        t, uni = runWaltersCode()
        print(z,x, "Time of Matthias Walter: ",sum(t), "Number of unimodular fields:", sum(uni))
        numUni[z].append(uni)
        timeWalter[z].append(t)

        #t = time.time()
        #for j in range(len(fields)): 
         #    ip(fields[j])
        #timeIP[z].append(time.time()-t)
        #print(z,x, "Time IP: ",timeIP[z])

        print("\n")

    print("done with ", x, "round!\n\n")
    
#print("our Algo")
#print(timeUs)
#print("time walter")
#print(timeWalter)


In [ ]:
sumW = [[0 for i in range(len(timeWalter[0][0]))] for j in range(len(timeWalter[0]))]
for i in range(len(timeWalter)):
    for j in range(len(timeWalter[i])):
        for jj in range(len(timeWalter[i][j])):
            sumW[j][jj] = sumW[j][jj] + timeWalter[i][j][jj]/wdh


sumU = [[0 for i in range(len(timeUs[0][0]))] for j in range(len(timeUs[0]))]
for i in range(len(timeUs)):
    for j in range(len(timeUs[i])):
        for jj in range(len(timeUs[i][j])):
            sumU[j][jj] = sumU[j][jj] + timeUs[i][j][jj]/wdh

In [ ]:
import pandas as pd
results = ["TU-Test 5x5", "TU-Test 10x10", "TU-Test 15x15", "TU-Test 20x20", "TU-Test 25x25", "Balancedness 5x5", "Balancedness 10x10", "Balancedness 15x15", "Balancedness 20x20", "Balancedness 25x25"]
res = [*sumW, *sumU]
np.save("results",res)
print(len(res[1]),len(res[7]))


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "matplotlib"
x = np.arange(0.000001,10.01,0.000001)
res = np.load("results.npy")
anz = 100

df=pd.DataFrame({"TU-Test 5x5": res[0], "TU-Test 10x10":res[1], "TU-Test 15x15":res[2], "TU-Test 20x20":res[3], "TU-Test 25x25":res[4], "Balancedness 5x5":res[5], "Balancedness 10x10":res[6], "Balancedness 15x15":res[7], "Balancedness 20x20":res[8], "Balancedness 25x25":res[9]})#print(df)
t5= np.sum(df["TU-Test 5x5"][None,:] <= x[:,None], axis = 1) / anz
t10= np.sum(df["TU-Test 10x10"][None,:] <= x[:,None], axis = 1) / anz
t15 = np.sum(df["TU-Test 15x15"][None,:] <= x[:,None], axis = 1) / anz

t20 = np.sum(df["TU-Test 20x20"][None,:] <= x[:,None], axis = 1) / anz
t25  = np.sum(df["TU-Test 25x25"][None,:] <= x[:,None], axis = 1) / anz
b5 = np.sum(df["Balancedness 5x5"][None,:] <= x[:,None], axis = 1) / anz
b10 = np.sum(df["Balancedness 10x10"][None,:] <= x[:,None], axis = 1) / anz
b15 = np.sum(df["Balancedness 15x15"][None,:] <= x[:,None], axis = 1) / anz
b20 = np.sum(df["Balancedness 20x20"][None,:] <= x[:,None], axis = 1) / anz

b25 = np.sum(df["Balancedness 25x25"][None,:] <= x[:,None], axis = 1) / anz
plot_DF = pd.DataFrame({"Timeout": x, "TU-Test 5x5":t5, "TU-Test 10x10":t10, "TU-Test 15x15":t15, "TU-Test 20x20":t20, "TU-Test 25x25":t25, "B-Test 5x5":b5, "B-Test 10x10":b10, "B-Test 15x15":b15, "B-Test 20x20":b20, "B-Test 25x25":b25})#, yticks = [0,0.1,0.2,0.3, 0.4,0.5,0.6,0.7,0.8,0.9,1] )#xlim = [0.01,0200])
#plot_DF.savefig('SeWaltervsBalancedng',dpi=600, bbox_inches = "tight")
plot_DF.plot(figsize=(5,4), color = ['black','black','black','black','black', 'grey', 'grey', 'grey', 'grey', 'grey'], style = {"TU-Test 5x5":"--", "TU-Test 15x15":"--", "TU-Test 25x25":"--","TU-Test 20x20":"--", "TU-Test 10x10":"--"}, x = "Timeout", logx = True, xlabel="Timeout (in seconds)", ylabel="Fraction of solved examples", ylim = [0,1], yticks=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.savefig('WaltervsBalanced.png',dpi=100, bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(figsize=(4,2))
#plt.fig.rcParams['fig.figsize'] =(5, 3)
r=np.arange(0, 6)
test=[r,r,r,r]
ax.plot(r,timeWalter,r,timeUs)
names=["Walter", "Our"]
ax.legend(names)
plt.show()

In [ ]:
data = [numUni, numBal, timeWalter, timeUs] 
names=["numUni","numBal", "timeWalter", "timeUs"]
fig = plt.figure(figsize =(4,2)) 
ax = fig.add_axes([0, 0, 1, 1]) 
bp = ax.boxplot(data, showfliers=False,patch_artist = True,vert = 0,labels=names) 
plt.show()

In [ ]:
#  array = range(25,26,5)

# anz = 100

# for j in array:
#     gg = []
#     matrices = []
#     for i in range(anz):
#         t = time.time()
#         g = []
#         h=[]
#         while len(h) == 0:
#             g = start(j, j)
#             h = computeIPMatrix(g)
            
#         saveMatrix(h,i)
#         gg.append(g)
#         matrices.append(h)
#         print(j,i,time.time()-t)
        
#     fileObj = open('fields%s.obj'%j, 'wb')
#     pickle.dump(gg,fileObj)
#     fileObj.close()
#     fileObj = open('matrices%s.obj'%j, 'wb')
#     pickle.dump(matrices,fileObj)
#     fileObj.close()

#     print("Size", j)

In [ ]:
def ip(field):
    w,h=len(field),len(field[0])
    rect=[]
    rectangles = [[[] for i in range(h)] for j in range(w)]
    for i in range(w):
        for j in range(h):
            if field[i][j]!=0:
                d=h
                for ii in range(i,w):
                    for jj in range(j,d):
                        if field[ii][jj]==0: 
                            d=jj
                            break
                        for iii in range(i,ii+1):
                            for jjj in range(j,jj+1):
                                rectangles[iii][jjj].append((i,j,ii,jj))
                        rect.append((i,j,ii,jj))
    m = Model()
    m.Params.OutputFlag=0
    rects = m.addVars(rect,vtype=GRB.BINARY,name='Rects')
    for i in range(w):
        for j in range(h):
            if g[i,j]!=0:
                m.addConstr(sum(rects[x] for x in rectangles[i][j])==1) 
    m.setObjective(sum(rects[x]*(x[2]-x[0]+1)*(x[3]-x[1]+1)*(x[2]-x[0]+1)*(x[3]-x[1]+1) for x in rect), GRB.MAXIMIZE)
    m.Params.Heuristics=0
    m.Params.Presolve=0
    m.optimize()
    return m.objVal